In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import tensorflow as tf
!pip install bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert: filename=bert-2.2.0-py3-none-any.whl size=3745 sha256=0fb874f00555cf946df1f00cc5a30e15dd52d3d31f17d02c332b3cfc1a52b364
  Stored in directory: /root/.cache/pip/wheels/d0/97/c8/5775d57c323970511488d9b53ce66cf0ded394fe4b4f6e0afb
  Created wheel for erlastic: filename=erlastic-2.0.0-py3-none-any.whl size=6777 sha256=eed9f9c8b4df290823ea8b02a50a21e30e12f085fd555e6c185759ee3d8fac24
  Stored in directory: /root/.cache/pip/wheels/f5/ca/b1/ce71c2e0e9e0df3786697d06d53574be03ef86cd4fb0e2cb99
Successfully built bert erlastic


In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
data = pd.read_csv('/content/drive/Shareddrives/MSBA Capstone Sharing/2. Data/230607 Events dump.csv')

Mounted at /content/drive


<ipython-input-3-047fd8e8cd66>:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/Shareddrives/MSBA Capstone Sharing/2. Data/230607 Events dump.csv')


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.8 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertModel

def bert_similarity(text1, text2):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertModel.from_pretrained("bert-base-uncased")

    input_ids = tokenizer.encode_plus(text1, text2, add_special_tokens=True, truncation=True, max_length=128, padding="longest", return_tensors="pt")["input_ids"]
    outputs = model(input_ids)
    embeddings = torch.mean(outputs.last_hidden_state, dim=1)

    # Pairwise cosine similarity
    num_rows = embeddings.shape[0]
    similarity_matrix = torch.nn.functional.cosine_similarity(embeddings.unsqueeze(1).expand(-1, num_rows, -1), embeddings.unsqueeze(0).expand(num_rows, -1, -1), dim=2)

    return similarity_matrix

similarity = bert_similarity("Text 1", "Text 2")
print(similarity)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[1.0000]], grad_fn=<SumBackward1>)


In [ ]:
import torch
from transformers import BertTokenizer, BertModel

def bert_similarity(text1, text2):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertModel.from_pretrained("bert-base-uncased")

    input_ids = tokenizer.encode_plus(text1, text2, add_special_tokens=True, truncation=True, max_length=128, padding="longest", return_tensors="pt")["input_ids"]
    print("Input IDs:", input_ids)

    outputs = model(input_ids)
    embeddings = torch.mean(outputs.last_hidden_state, dim=1)
    print("Embeddings:", embeddings)

    # Pairwise cosine similarity for all embeddings
    num_rows = embeddings.shape[0]
    similarity_matrix = torch.nn.functional.cosine_similarity(embeddings.unsqueeze(1).expand(-1, num_rows, -1), embeddings.unsqueeze(0).expand(num_rows, -1, -1), dim=2)
    print("Similarity Matrix:", similarity_matrix)

    return similarity_matrix

similarity = bert_similarity("Text 1", "Text 2")
print(similarity)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Input IDs: tensor([[ 101, 3793, 1015,  102, 3793, 1016,  102]])
Embeddings: tensor([[ 2.0882e-02,  2.3185e-02, -1.7139e-02, -2.2950e-01,  1.7403e-02,
         -2.2704e-01,  2.9631e-01,  4.7146e-02,  1.8751e-01, -2.1707e-01,
         -1.2075e-01,  5.5599e-02, -1.5495e-02, -3.8532e-02, -4.4182e-01,
          4.0875e-02,  7.4846e-02,  3.2770e-01,  3.4013e-03,  4.0618e-01,
          2.9285e-01,  3.1626e-02, -8.9187e-02,  2.6636e-01,  3.3744e-01,
          1.8519e-01, -2.2406e-01, -2.3816e-01,  3.3685e-02,  5.3851e-02,
          1.1472e-01, -4.4080e-01, -3.5321e-02,  3.0891e-01,  9.5720e-03,
         -9.9824e-02, -2.4672e-01, -2.0159e-01,  6.1744e-02,  8.8708e-02,
         -2.8548e-01, -2.0705e-01, -2.0137e-01, -3.5894e-01,  3.0243e-01,
         -4.8498e-02, -2.3110e-01,  2.0350e-01, -4.7067e-01,  2.2660e-01,
         -1.9618e-01,  2.1369e-01, -3.3169e-01,  4.4831e-01,  1.1936e-01,
          1.7999e-01,  1.2974e-01,  4.6712e-03, -3.8948e-01, -2.8632e-02,
          2.2543e-01,  1.2648e-01, -

In [ ]:
def find_duplicates(data, threshold=0.8):
    """Finds duplicates in the data based on BERT similarity and adds a 'duplicate' column."""
    duplicates = set()
    for i in range(len(data)):
        if data[i]["summary"] is None:
            continue
        for j in range(i + 1, len(data)):
            if data[j]["summary"] is None:
                continue
            similarity = bert_similarity(data[i]["summary"], data[j]["summary"])
            if similarity > threshold:
                duplicates.add(i)
                duplicates.add(j)

    for i in range(len(data)):
        data[i]["duplicate"] = 1 if i in duplicates else 0
    return data
if __name__ == "__main__":
    updated_data = find_duplicates(data, 0.8)
    for row in updated_data:
        print(row)